# Emulating the ROM bootloader

This notebook is a basic example to show how the extracted ROM bootloader can be emulated using [Unicorn engine](https://github.com/unicorn-engine/unicorn).

You will have to first extract the ROM bootloader from a CC2640R2F, as documented in [1_extract_rom_bootloader.ipynb](). We also recommend that you follow the instructions to load the ROM bootloader in Ghidra. While attempting to emulate the ROM bootloader it will be helpfull to have the Ghidra disassembly/decompilation to see where the emulation gets stuck or fails.

Let's start by importing all the required libraries and loading the ROM bootloader.

In [1]:
from unicorn import *
from unicorn.arm_const import *
from capstone import *
import time
from tqdm.notebook import tqdm
import numpy as np

In [2]:
with open('./rom_bootloader_cc2640r2.bin', 'rb') as f:
    bootrom = f.read()

## A first attempt

Emulating a binary in Unicorn engine is relatively straightforward, we start by creating the emulator (ARM THUMB in this case). Afterwards we map a memory region that will hold the code to be executed, and we fill that memory region with the code. In this case we map a memory region starting at `BOOTROM_START_ADDRESS` and write the extracted ROM bootloder to that memory region. 

Afterwards we start the emulator and instruct it to start executing instructions starting from the main entrypoint we identified in Ghidra.

In [3]:
BOOTROM_START_ADDRESS = 0x10000000
BOOTROM_ENTRYPOINT = 0x1000162b

In [4]:
# Create an ARM THUMB emulator
mu = Uc(UC_ARCH_ARM, UC_MODE_THUMB)

# map and write bootrom
mu.mem_map(BOOTROM_START_ADDRESS, len(bootrom))
mu.mem_write(BOOTROM_START_ADDRESS, bootrom)

# Start the emulation
mu.emu_start(BOOTROM_ENTRYPOINT, 0x100010aa, timeout=1000000)

UcError: Invalid memory write (UC_ERR_WRITE_UNMAPPED)

In [5]:
pc = mu.reg_read(UC_ARM_REG_PC)
print('Current PC:', hex(pc))

Current PC: 0x1000162a


The emulation fails and Unicorn throws an `UC_ERR_WRITE_UNMAPPED` exception. Meaning that the last instruction we were trying to execute tried to perform a write to an unmapped memory region. This is unsurprising as we have not mapped all of the other memory regions (recall the memory map from the Ghidra instructions) that may be required to execute the ROM bootloader. 

## Mapping memory regions
In Ghidra we used the [SVD-Loader](https://github.com/leveldown-security/SVD-Loader-Ghidra) plugin to automatically populate the memory map, here we will apply the same idea to our unicorn emulation. We will parse a SVD file and map  all memory regions.

To do this we will use the [CMSIS SVD Parser](https://github.com/posborne/cmsis-svd/tree/master/python) and an [SVD file](https://github.com/posborne/cmsis-svd/blob/master/data/TexasInstruments/CC26x0.svd) for our target microcontroller.

In [ ]:
!wget -nv https://github.com/posborne/cmsis-svd/raw/master/data/TexasInstruments/CC26x0.svd

In [6]:
from cmsis_svd.parser import SVDParser

parser = SVDParser.for_xml_file('./CC26x0.svd')
device = parser.get_device()

print(device.cpu.name, device.cpu.endian)

for peripheral in device.peripherals:
    length = peripheral.address_block.offset + peripheral.address_block.size
    print("%s @ 0x%08x" % (peripheral.name, peripheral.base_address))

CM3 little
AON_BATMON @ 0x40095000
AON_EVENT @ 0x40093000
AON_IOC @ 0x40094000
AON_RTC @ 0x40092000
AON_SYSCTL @ 0x40090000
AON_WUC @ 0x40091000
AUX_ADI4 @ 0x400cb000
AUX_AIODIO0 @ 0x400c1000
AUX_AIODIO1 @ 0x400c2000
AUX_ANAIF @ 0x400c9000
AUX_DDI0_OSC @ 0x400ca000
AUX_EVCTL @ 0x400c5000
AUX_SCE @ 0x400e1000
AUX_SMPH @ 0x400c8000
AUX_TDCIF @ 0x400c4000
AUX_TIMER @ 0x400c7000
AUX_WUC @ 0x400c6000
CCFG @ 0x50003000
CPU_TIPROP @ 0xe00fe000
CRYPTO @ 0x40024000
EVENT @ 0x40083000
FCFG1 @ 0x50001000
FLASH @ 0x40030000
GPIO @ 0x40022000
GPT0 @ 0x40010000
GPT1 @ 0x40011000
GPT2 @ 0x40012000
GPT3 @ 0x40013000
I2C0 @ 0x40002000
I2S0 @ 0x40021000
IOC @ 0x40081000
PRCM @ 0x40082000
RFC_DBELL @ 0x40041000
RFC_PWR @ 0x40040000
RFC_RAT @ 0x40043000
SMPH @ 0x40084000
SSI0 @ 0x40000000
SSI1 @ 0x40008000
TRNG @ 0x40028000
UART0 @ 0x40001000
UDMA0 @ 0x40020000
VIMS @ 0x40034000
WDT @ 0x40080000


We can use the information in the SVD file to automatically map these peripheral memory regions.
Additionally we have to map a few memory regions ourselves (e.g. the SRAM).

We also add a basic callback function that prints information on the executed instructions/basic blocks.

In [7]:
BOOTROM_START_ADDRESS = 0x10000000
BOOTROM_ENTRYPOINT = 0x1000162b
FLASH_START_ADDRESS = 0x00000000
SRAM_START_ADDRESS = 0x20000000
CACHE_START_ADDRESS = 0x11000000 # Memory block is called cache in the memory map but appears to be used as ram

In [8]:
cs = Cs(CS_ARCH_ARM, CS_MODE_THUMB) # We use capstone for dissasembly

def hook_block(uc, address, size, user_data):
    print("Executing basic block at 0x%x, block size = 0x%x" %(address, size))

def hook_code(uc, address, size, user_data):
    code = uc.mem_read(address, 4)
    for ins in cs.disasm(code, 1):
        print("0x%x: %s\t\t%s\t%s" %(address, ins.bytes.hex(), ins.mnemonic, ins.op_str))  

In [9]:
# Create an ARM THUMB emulator
mu = Uc(UC_ARCH_ARM, UC_MODE_THUMB)

# map and write bootrom
mu.mem_map(BOOTROM_START_ADDRESS, len(bootrom))
mu.mem_write(BOOTROM_START_ADDRESS, bootrom)

# map all peripherals
for peripheral in device.peripherals:
    length = peripheral.address_block.offset + peripheral.address_block.size
    try:
        mu.mem_map(peripheral.base_address, length)
    except:
        #print('Failed to map', peripheral.name) 
        continue
        
# manual map
mu.mem_map(0x04000000, 0x1000)
mu.mem_map(0xe0000000, 0xF000) # Not in the SVD file but part of the peripheral memory region
mu.mem_map(SRAM_START_ADDRESS, 0x5000)
mu.mem_map(CACHE_START_ADDRESS, 0x2000)
mu.reg_write(UC_ARM_REG_SP, CACHE_START_ADDRESS+0x2000)

# add hooks
#mu.hook_add(UC_HOOK_CODE, hook_code)
mu.hook_add(UC_HOOK_BLOCK, hook_block)

# Start the emulation
mu.emu_start(BOOTROM_ENTRYPOINT, 0x100010aa, timeout=20000)

pc = mu.reg_read(UC_ARM_REG_PC)
print('Current PC:', hex(pc))

Executing basic block at 0x1000162a, block size = 0x28
Executing basic block at 0x10001690, block size = 0x1c
Executing basic block at 0x100016f0, block size = 0xc
Executing basic block at 0x1000174c, block size = 0x1a
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 0x1000175c, block size = 0xa
Executing basic block at 

As you can see from the above output, the emulation now executes some instructions and then gets stuck in an endless loop. At this point we have to start "massaging" the execution. We can do this by looking at the code in Ghidra and identifying why the execution gets stuck. Having identified the issue we can probably work our way around it by adding an additional hook or callback to the Unicorn emulator.

In this case the execution gets stuck at 0x1000175c. If we look at the ROM bootloader code in Ghidra we can see that the code is polling the `FLASH:EFUSEERROR` register waiting for the 6th bit to be cleared.

```c
  write_volatile_4(Peripherals::FLASH.EFUSEREAD,0x50);
  write_volatile_4(Peripherals::FLASH.EFUSEERROR,0);
  write_volatile_4(Peripherals::FLASH.EFUSEADDR,0);
  write_volatile_4(Peripherals::FLASH.EFUSE,0x400000a);
  do {
    uVar3 = read_volatile_4(Peripherals::FLASH.EFUSEERROR);
  } while ((uVar3 & 0x20) == 0);
```

To understand what is going on here we can look at some of the documentation provided by Texas Instruments.
First, the [`FLASH` peripheral register description](https://software-dl.ti.com/simplelink/esd/simplelink_cc2640r2_sdk/5.30.00.03/exports/docs/driverlib_cc13xx_cc26xx/cc26x0r2/register_descriptions/CPU_MMAP/FLASH.html) can provide us information about the eFuse related registers. Second, we can look at the [`FlashEfuseReadRow` function](https://software-dl.ti.com/simplelink/esd/simplelink_cc2640r2_sdk/5.30.00.03/exports/docs/driverlib_cc13xx_cc26xx/cc26x0r2/driverlib/group__flash__api.html#gacecbeef915333831a97d054df9e8fc40) provided as part of the driver library.
You should be able to see some similarities between the `FlashEfuseReadRow` function and the ROM bootloader code. In the `FlashEfuseReadRow` function there is a write to the `FLASH:EFUSEREAD`, followed by writes to the `FLASH:EFUSEERROR`, `FLASH:EFUSEADDR`, and `FLASH:EFUSE` registers. Afterwards there is a while loop, waiting for the `DONE` bit of the `FLASH:EFUSEERROR` register to be set.

The emulation of the ROM bootloader is stuck at exactly this part of the execution, because there is no eFuse peripheral that will write a 1 to the `DONE` bit of the `FLASH:EFUSEERROR` register, signaling that the eFuse read is completed.

To emulate the eFuse reads correctly we used the `FlashEfuseReadRow` on our LAUNCHXL-CC2640R2 development board to read all eFuse rows and implemented a simple callback function to handle eFuse reads.

In [10]:
p_flash_efuseerror = 0x4003103c
p_flash_efuse = 0x40031000
p_flash_datalower = 0x4003100C

# Dumped from a LAUNCHXL-CC2640R2 development board using FlashEfuseReadRow
efuses = b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6E\x00`\x00\x00\x00\x00\x10\xc0h'\x88 \x0c\x90\x01\x00\x00\x00\x00\x00\x00\x00\x05\x01\x80\xc0\xaa\x0c\x0f+\xe3\xff\xf8\x01\x82k\x03\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6E\x00`\x00\x00\x00\x00\x10\xc0h'\x88 \x0c\x90\x01\x00\x00\x00\x00\x00\x00\x00\x05\x01\x80\xc0\xaa\x0c\x0f+\xe3\xff\xf8\x01\x82k\x03\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

def hook_efuse(uc, access, address, size, value, bkptno):
    pc = uc.reg_read(UC_ARM_REG_PC)
    print('Hooked EFUSEERROR read 0x%08x' % pc)
    
    # Set the `DONE` bit in the FLASH:EFUSEERROR register
    uc.mem_write(p_flash_efuseerror, bytes([0x20,0,0,0]))
    
    # The FLASH:EFUSE register contains the row that is being read
    addr = uc.mem_read(p_flash_efuse, 4)
    addr = int.from_bytes(addr, 'little') & 0xFFFF
    print('\tTrying to read EFUSE row', addr, hex(addr))
    
    # Get the data of that row and write it to the FLASH:DATALOWER register
    efuse_dat = efuses[addr*4:addr*4 + 4]
    
    print('\tEfuse value:', efuse_dat.hex())
    uc.mem_write(p_flash_datalower, efuse_dat)

In [11]:
# Create an ARM THUMB emulator
mu = Uc(UC_ARCH_ARM, UC_MODE_THUMB)

# map and write bootrom
mu.mem_map(BOOTROM_START_ADDRESS, len(bootrom))
mu.mem_write(BOOTROM_START_ADDRESS, bootrom)

# map all peripherals
for peripheral in device.peripherals:
    length = peripheral.address_block.offset + peripheral.address_block.size
    try:
        mu.mem_map(peripheral.base_address, length)
    except:
        #print('Failed to map', peripheral.name) 
        continue
        
# manual map
mu.mem_map(0x04000000, 0x1000)
mu.mem_map(0xe0000000, 0xF000) # Not in the SVD file but part of the peripheral memory region
mu.mem_map(SRAM_START_ADDRESS, 0x5000)
mu.mem_map(CACHE_START_ADDRESS, 0x2000)
mu.reg_write(UC_ARM_REG_SP, CACHE_START_ADDRESS+0x2000)

# add code hooks
#mu.hook_add(UC_HOOK_CODE, hook_code)
mu.hook_add(UC_HOOK_BLOCK, hook_block)

# add efuse hook
# We want our callback function to run when the code reads the FLASH:EFUSEERROR register
mu.hook_add(UC_HOOK_MEM_READ, hook_efuse, begin=p_flash_efuseerror, end=p_flash_efuseerror)

# Start the emulation
mu.emu_start(BOOTROM_ENTRYPOINT, 0x100010aa, timeout=20000)

pc = mu.reg_read(UC_ARM_REG_PC)
print('Current PC:', hex(pc))

Executing basic block at 0x1000162a, block size = 0x28
Executing basic block at 0x10001690, block size = 0x1c
Executing basic block at 0x100016f0, block size = 0xc
Executing basic block at 0x1000174c, block size = 0x1a
Hooked EFUSEERROR read 0x1000174c
	Trying to read EFUSE row 10 0xa
	Efuse value: aa0c0f2b
Executing basic block at 0x10001766, block size = 0xa
Hooked EFUSEERROR read 0x10001766
	Trying to read EFUSE row 10 0xa
	Efuse value: aa0c0f2b
Executing basic block at 0x10001774, block size = 0xa
Executing basic block at 0x1000177e, block size = 0xa
Executing basic block at 0x10001788, block size = 0x2
Executing basic block at 0x1000178e, block size = 0x28
Executing basic block at 0x100017b6, block size = 0x8
Executing basic block at 0x100017b6, block size = 0x8
Executing basic block at 0x100017b6, block size = 0x8
Executing basic block at 0x100017b6, block size = 0x8
Executing basic block at 0x100017b6, block size = 0x8
Executing basic block at 0x100017b6, block size = 0x8
Execut

Now the execution proceeds, and we can see that the ROM bootloader reads the same efuse row twice.
Unfortunately the execution still gets stuck, so we have to continue guiding the execution.

## Full execution
With some effort you could end up with something as in the following cells. We added some more hooks and memory regions but now the ROM bootloader code executes up to the point when JTAG is being enabled/disabled.

In [12]:
BOOTROM_START_ADDRESS = 0x10000000
FLASH_START_ADDRESS = 0x00000000
SRAM_START_ADDRESS = 0x20000000
CACHE_START_ADDRESS = 0x11000000

BOOTROM_ENTRYPOINT = 0x1000162b

p_flash_efuseerror = 0x4003103c
p_flash_efuse = 0x40031000
p_flash_datalower = 0x4003100C
p_aonwuc_pwrstat = 0x40091014
p_aonwuc_jtagcfg = 0x40091040
p_prcm_clkloadctl = 0x40082028

# Dumped from a LAUNCHXL-CC2640R2 development board using FlashEfuseReadRow
efuses = b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6E\x00`\x00\x00\x00\x00\x10\xc0h'\x88 \x0c\x90\x01\x00\x00\x00\x00\x00\x00\x00\x05\x01\x80\xc0\xaa\x0c\x0f+\xe3\xff\xf8\x01\x82k\x03\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6E\x00`\x00\x00\x00\x00\x10\xc0h'\x88 \x0c\x90\x01\x00\x00\x00\x00\x00\x00\x00\x05\x01\x80\xc0\xaa\x0c\x0f+\xe3\xff\xf8\x01\x82k\x03\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

# CCFG and FCFG example data
fcfg_data = b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xae\xac+\x0b\x80\xca\x02\x00\xff\xff\xff\xff\xff\xff\xff\xff\xa6\xbc+\x0b\x00\x00\xa8\xb2\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\xff\xff\xff\xff\xff\xff\xffa\x05\x89\x02\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6E\x00`\x00\x00\x00\x00\x10\xc0h'\x88 \x0c\x90\x01\x00\x00\x00\x00\x00\x00\x00\x05\x01\x80\xc0\xaa\x0c\x0f+\xe3\xf0\xf8\x01\x82k\x03\x02\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xd5\x99\x88g\xff\xff\xff\xff\xff\xff\xff\xff\x00\xff\xff\xffh$\x14\x00\x17\x00\xbe\x7fg\x00\t\x003\x1a3\x17\x00 \n\n\x00\x02n\x02\x00\xf0\x00\x02\x01\x00\x00\x00\x14\x00\x05\x00\x10\x00\xa0\x0f\xa0\x0f\x00\x00\x04\n\x02\x0b\x01\x08\x08\x08\x08\x07\n\x08j\xa5\xe0\x99\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x01\x00\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfe\xff\xff\xff\x00\x90\x02\xb0\xc5\x00\xc5\xff\x00\x00\x00\xff\x00\xc5\xf2\xff\xff\xff\xff\xff\x03\xe7\x03\x00\xf3///\x03 \x11\x00\x7f\xf4M\xcb\xfb\xe0\xf8\xfa\xfc\xf0\xff\xf1\xff\xf7\xff\xe8\xaf\x8dp\x92\x0b\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfd\xff\xff\xff\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xddkG\xcfa\x0c\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xbc\x13\x00\x00\xff\x13\x00\x00\xff\xff\xff\xff\x07\xfe\x07\xfe\x9f\x9f\x98\x983\xff\xff\xff\x03\x00\x00\x00\x11\x02\x11\x02/\xa0\x99\xbb%\x00\x00\x00\n\xc6\x00\x00\x7f\x91\x90\x8f\x8e\x8d\x8c\x8b\x8a\x89\x88\x87\x86\x85\x84\x83\x82\x81\x80\x9e\x9d\x9c\x9b\x9a\x99\x98\x97\x96\x95\x94\x93\x92\x1f\xff\xff\xffb,\x95\xd0\xdd\xf7`4T\xff\x00\xfc\xe7_\x03x\xfa\x0e\x00\xf8\xcd\x81\xff\xff\xc8\x83\xff\xff\x00\x00\xff\xff\xfb\xff\xfa\xff\x89\xff\x00\xff\x8e\x82{\xff\x03\x03\x8bkG?\x18\xff\xff\xc3\xff\xffM\xff\xfd\xff\x1eU\x1e\xff\xfd\xff\xff\xff\x18\x00\x0e\xf0\x00\x00\x00\x00\xff\xff\xff\xff\xf8\x03\x04\xe0\x08\xa6\x0b\x08\n\xa5\x10\x0c\r\xa2\x18\x12\x14\x9c%\x1c!\x90;.;zbLkp\x9ex\x9a\x80\xe1\xad\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
ccfg_data = b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x80\x01\x10\x00\x82\xff\xfd\xffX\x00:\xff\xbf\xf3\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\xff\xff\xff\xff\x00\xff\xff\xff\xc5\x00\xc5\xff\x00\x00\x00\xff\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'

In [13]:
cs = Cs(CS_ARCH_ARM, CS_MODE_THUMB) # We use capstone for dissasembly

def hook_block(uc, address, size, user_data):
    print("Executing basic block at 0x%x, block size = 0x%x" %(address, size))

def hook_code(uc, address, size, user_data):
    code = uc.mem_read(address, 4)
    for ins in cs.disasm(code, 1):
        print("0x%x: %s\t\t%s\t%s" %(address, ins.bytes.hex(), ins.mnemonic, ins.op_str))      

def hook_efuse(uc, access, address, size, value, bkptno):
    pc = uc.reg_read(UC_ARM_REG_PC)
    print('Hooked EFUSEERROR read @ 0x%08x' % pc)
    
    # Set the `DONE` bit in the FLASH:EFUSEERROR register
    uc.mem_write(p_flash_efuseerror, bytes([0x20,0,0,0]))
    
    # The FLASH:EFUSE register contains the row that is being read
    addr = uc.mem_read(p_flash_efuse, 4)
    addr = int.from_bytes(addr, 'little') & 0xFFFF
    print('\tTrying to read EFUSE row', addr, hex(addr))
    
    # Get the data of that row and write it to the FLASH:DATALOWER register
    efuse_dat = efuses[addr*4:addr*4 + 4]
    
    print('\tEfuse value:', efuse_dat.hex())
    uc.mem_write(p_flash_datalower, efuse_dat)
    
def hook_aonwuc_pwrstat(uc, access, address, size, value, bkptno):
    print('Hooked aon_wuc:pwrstat read @ 0x%08x' % address)
    uc.mem_write(p_aonwuc_pwrstat, bytes([0x20,0,0,0]))

def hook_prcm_clkloadctl(uc, access, address, size, value, bkptno):
    print('Hooked PRCM:CLKLOADCTL read @ 0x%08x' % address)
    uc.mem_write(p_prcm_clkloadctl, bytes([2,0,0,0])) 
    
def hook_fcfg(uc, access, address, size, value, bkptno):
    ret = uc.mem_read(address, 4)
    print('Hooked FCFG read @ 0x%08x, (0x%s)' % (address, ret.hex()))
    
def hook_ccfg(uc, access, address, size, value, bkptno):
    ret = uc.mem_read(address, 4)
    print('Hooked CCFG read @ 0x%08x (0x%s)' % (address, ret.hex()))
    
def hook_aon_wuc_jtagcfg(uc, access, address, size, value, bkptno):
    print('Hooked write to AON_WUC:JTAGCFG 0x%08x, 0x%08x, 0x%08x' % (access, address, value))
    
def hook_skip4(uc, address, size, user_data):
    pc = uc.reg_read(UC_ARM_REG_PC)
    #print('skipping at %08x, %08x' % (pc, pc+4))
    uc.reg_write(UC_ARM_REG_PC, pc+4)
    cpsr = uc.reg_read(UC_ARM_REG_CPSR) | 0x20
    uc.reg_write(UC_ARM_REG_CPSR, cpsr)

In [14]:
# Create an ARM THUMB emulator
mu = Uc(UC_ARCH_ARM, UC_MODE_THUMB)

# map and write bootrom
mu.mem_map(BOOTROM_START_ADDRESS, len(bootrom))
mu.mem_write(BOOTROM_START_ADDRESS, bootrom)

# map all peripherals
for peripheral in device.peripherals:
    length = peripheral.address_block.offset + peripheral.address_block.size
    try:
        mu.mem_map(peripheral.base_address, length)
    except:
        #print('Failed to map', peripheral.name) 
        continue
        
# manual map
mu.mem_map(0x04000000, 0x1000)
mu.mem_map(0x40086000, 0x1000)
mu.mem_map(0x400ca000, 0x1000)

mu.mem_map(0xe0000000, 0xF000) # Not in the SVD file but part of the peripheral memory region
mu.mem_map(SRAM_START_ADDRESS, 0x5000)
mu.mem_map(CACHE_START_ADDRESS, 0x2000)
mu.reg_write(UC_ARM_REG_SP, CACHE_START_ADDRESS+0x2000)

# write FCFG and CCFG
mu.mem_map(0x50001400, 0x400)
mu.mem_write(0x50001000, fcfg_data)
mu.mem_write(0x50003000, ccfg_data)

# set prcm pdstat0
mu.mem_write(0x40082140, bytes([4, 0, 0, 0]))
mu.mem_write(0x40082028, bytes([2, 0, 0, 0]))

mu.mem_write(0x4003001c, bytes([4, 0, 0, 0])) # set FLASH:STAT
mu.mem_write(0x4003002c, b' \x00\x00\x00') # set FLASH:FLASH_SIZE
mu.mem_write(0x40032430, b'\x04\x00 \x00') # set FLASH.FCFG_B0_SSIZE0       

# add code hooks
#mu.hook_add(UC_HOOK_CODE, hook_code)
#mu.hook_add(UC_HOOK_BLOCK, hook_block)

# add efuse hook
# We want our callback function to run when the code reads the FLASH:EFUSEERROR register
mu.hook_add(UC_HOOK_MEM_READ, hook_efuse, begin=p_flash_efuseerror, end=p_flash_efuseerror)

mu.hook_add(UC_HOOK_MEM_READ, hook_aonwuc_pwrstat, begin=p_aonwuc_pwrstat, end=p_aonwuc_pwrstat)
mu.hook_add(UC_HOOK_MEM_READ, hook_fcfg, begin=0x50001000, end=0x50001000+0x400)
mu.hook_add(UC_HOOK_MEM_READ, hook_ccfg, begin=0x50003000, end=0x50003000+0x1000)
mu.hook_add(UC_HOOK_MEM_READ, hook_prcm_clkloadctl, begin=p_prcm_clkloadctl, end=p_prcm_clkloadctl)
mu.hook_add(UC_HOOK_MEM_WRITE, hook_aon_wuc_jtagcfg, begin=p_aonwuc_jtagcfg, end=p_aonwuc_jtagcfg)
mu.hook_add(UC_HOOK_CODE, hook_skip4, begin=0x1000178a, end=0x1000178a)
mu.hook_add(UC_HOOK_CODE, hook_skip4, begin=0x10001bce, end=0x10001bce)
mu.hook_add(UC_HOOK_CODE, hook_skip4, begin=0x10001c30, end=0x10001c30)
mu.hook_add(UC_HOOK_CODE, hook_skip4, begin=0x100014b8, end=0x100014b8)

# Start the emulation
mu.emu_start(BOOTROM_ENTRYPOINT, 0x100010aa, timeout=2000000)

pc = mu.reg_read(UC_ARM_REG_PC)
print('Current PC:', hex(pc))

Hooked EFUSEERROR read @ 0x1000174c
	Trying to read EFUSE row 10 0xa
	Efuse value: aa0c0f2b
Hooked EFUSEERROR read @ 0x10001766
	Trying to read EFUSE row 10 0xa
	Efuse value: aa0c0f2b
Hooked aon_wuc:pwrstat read @ 0x40091014
Hooked EFUSEERROR read @ 0x100017be
	Trying to read EFUSE row 12 0xc
	Efuse value: 826b0302
Hooked EFUSEERROR read @ 0x100017e0
	Trying to read EFUSE row 12 0xc
	Efuse value: 826b0302
Hooked EFUSEERROR read @ 0x1000185c
	Trying to read EFUSE row 11 0xb
	Efuse value: e3fff801
Hooked EFUSEERROR read @ 0x10001888
	Trying to read EFUSE row 11 0xb
	Efuse value: e3fff801
Hooked EFUSEERROR read @ 0x100018da
	Trying to read EFUSE row 12 0xc
	Efuse value: 826b0302
Hooked EFUSEERROR read @ 0x100018f2
	Trying to read EFUSE row 12 0xc
	Efuse value: 826b0302
Hooked EFUSEERROR read @ 0x10001992
	Trying to read EFUSE row 11 0xb
	Efuse value: e3fff801
Hooked EFUSEERROR read @ 0x100019a0
	Trying to read EFUSE row 11 0xb
	Efuse value: e3fff801
Hooked EFUSEERROR read @ 0x100019da
	Tr

## Why bother?

That was quite some effort just to have the bootloader execute in an emulator.

You can use this example as a starting point to:

* Create side-channel traces by keeping track of memory read/writes and register updates.
  * You can use such a simulated side-channel trace and compare it with a real one to roughly determine a suitable glitch offset.
  * Have a look at Ledger's [Rainbow](https://github.com/Ledger-Donjon/rainbow) as an example.
  
* Simulate the effects of fault injection
  * You can implement a simple fault model (e.g. instruction skip) and see how this affects the execution.
  * You can also try to port this example to work with Riscure's [FiSim](https://github.com/Riscure/FiSim).
  
* Fuzzing of the serial bootloader interface
  * Maybe the ROM bootloader contains a software vulnerability?
  * You can try to fuzz the bootloader serial interface using [AFL++ in Unicorn mode](https://github.com/AFLplusplus/AFLplusplus/tree/stable/unicorn_mode).
  
* Record code coverage information
  * You can register a simple callback function that creates an execution trace.
  * Such a trace can be used to augment the Ghidra listing and decompiler views using [DragonDance](https://github.com/0ffffffffh/dragondance) or [Emerald](https://github.com/reb311ion/emerald).
  * Have a look at [Qiling's coverage extension](https://github.com/qilingframework/qiling/blob/master/qiling/extensions/coverage/formats/drcov.py) if you want to implement this.

![Ghidra Emerald](img/ghidra_emerald.png)